# Generate your predictions on test data and save the results in required format

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# define the ResNet model
class ResNetCIFAR(nn.Module):
    def __init__(self, ):
        super(ResNetCIFAR, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,kernel_size=5, stride=1, out_channels=8, padding=0)
        self.conv2 = nn.Conv2d(in_channels=8,kernel_size=3, stride=1, out_channels=16, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flat_features = 6 * 6 * 16
        self.fc1 = nn.Linear(self.flat_features, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.flat_features)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [2]:
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from tools.dataset import CIFAR10

DATA_ROOT = "./data"
BATCH_SIZE = 100

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_set = CIFAR10(
    root=DATA_ROOT, 
    mode='test', 
    download=True,
    transform=transform_test
)

# do NOT shuffle your test data loader!!!!!!!!!!!!!!!!
# otherwise the order of samples will be messed up
# and your test accuracy is likely to drop to random guessing level
test_loader = DataLoader(
    test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)

#########################################################
# use your model to generate predictions on test data
# and save the results into variable "results"
# "results" should be either a numpy array or a torch tensor with length of 10000

# initialize a resnet and load trained weights
net = ResNetCIFAR()
state_dict = torch.load('./saved_model/resnet.pth') # change the path to your own checkpoint file
net.load_state_dict(state_dict)
net.cuda()

# remember to switch to eval mode whenever you are making inference
net.eval()

results = []
with torch.no_grad():
    for x in test_loader:
        results.append(
            net(x.cuda()).argmax(1)
        )

# convert results to numpy array
results = torch.cat(results).cpu().numpy()
assert len(results) == 10000

#########################################################
with open('predictions.csv', 'w') as fp:
    fp.write("Id,Label\n")
    for i in range(len(results)):
        fp.write("%d,%d\n" %(i, results[i]))

28393472it [00:03, 9029485.71it/s]                               
C:\Users\kojil\AppData\Local\Temp\ipykernel_13876\1322157106.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feat

Extracting ./data\cifar10_test_F22.zip to ./data
Files already downloaded and verified


FileNotFoundError: [Errno 2] No such file or directory: './saved_model/resnet.pth'